In [39]:
!pip install langchain-openai
!pip install langgraph

## Setting up LLM

In [45]:
llm = ChatOpenAI(
        model="Qwen/Qwen3-4B",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key="abc-123",  # if you prefer to pass api key in directly instaed of using env vars
        base_url="http://localhost:8000/v1/",

        # organization="...",
        # other params...
        
    )

In [46]:
llm.invoke("Hello")

AIMessage(content='<think>\nOkay, the user said "Hello". I need to respond appropriately. Since they just greeted me, I should respond with a friendly greeting. Maybe say "Hello!" and ask how I can assist them. Keep it simple and welcoming. No need for any complex information here. Just a basic, polite response.\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 9, 'total_tokens': 87, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-4B', 'system_fingerprint': None, 'id': 'chatcmpl-ee8afce550ad4c51ae0fb031a9c5b50f', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--5db884ed-c7b2-4a25-8e3a-3e72ed8b5b7e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 78, 'total_tokens': 87, 'input_token_details': {}, 'output_token_details': {}})

## Set up LangGraph

In [36]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Annotated
import operator
import json
import requests
from datetime import datetime, timedelta

# Import necessary Google API client libraries - assuming they are installed
try:
    from google.oauth2.credentials import Credentials
    from googleapiclient.discovery import build
except ImportError:
    print("Google API client libraries not found. Calendar event retrieval will not function.")
    Credentials = None
    build = None

# --- 1. Define the State of your Agent ---
class AgentState(TypedDict):
    """
    Represents the state of our agent in the graph.
    """
    request_id: str
    datetime_of_request: str
    sender_email: str
    attendees_emails: List[str]
    subject: str
    email_content: str
    calendar_events: List[dict]  # To store fetched calendar events for all attendees
    event_start: str
    event_end: str
    duration_mins: str
    metadata: dict
    intermediate_steps: List[str] # To log thought process or actions
    tool_output: str # Output from tools
    llm_response: str # Final response from LLM or intermediate LLM thoughts
    # Added for the new meeting event details if scheduled
    scheduled_event_summary: str
    scheduled_event_attendees: List[str]
    location: str # Added location to AgentState


# --- 2. Define Tools (Functions the AI Agent can use) ---

# Provided Google Calendar Event Fetcher code
def retrive_calendar_events(user, start, end):
    ''' Use this tool to fetch the availabilty for the user on the Calendar.
    
        Args: 
          user: str (email-id of the user)
          start: datetime (Start of the event time)
          end: datetime (End of the event time)'''
    
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')

    count=0
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            events_list.append(
                {"StartTime" : start_time, 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(attendee_list)), 
                 "Attendees" : list(set(attendee_list)),
                 "Summary" : event["summary"]})
        except Exception as E:
            count+=1
    print('No of exceptions are: ',count)
    return events_list

# Tool 1: Google Calendar Event Fetcher
def fetch_calendar_events_tool(attendee_email: str, start_date_str: str, end_date_str: str) -> List[dict]:
    """
    Fetches calendar events for a given attendee within a date range using the provided retrieval function.
    """
    print(f"DEBUG: Calling fetch_calendar_events_tool for {attendee_email} from {start_date_str} to {end_date_str}")
    
    # Use the actual retrieval function directly
    retrieved_events = retrive_calendar_events(attendee_email, start_date_str, end_date_str)
    
    # Deduplicate events based on start, end, summary, and attendees
    unique_events = []
    seen = set()
    for event in retrieved_events:
        # Create a hashable tuple from event details to check for duplicates
        # Convert list of attendees to sorted tuple to ensure consistent hashing regardless of order
        event_tuple = (event.get('StartTime'), event.get('EndTime'), event.get('Summary'), tuple(sorted(event.get('Attendees', []))))
        if event_tuple not in seen:
            unique_events.append(event)
            seen.add(event_tuple)

    return unique_events


# Tool 2: Meeting Scheduler (Hypothetical - this would interact with Google Calendar API to create an event)
def schedule_meeting_tool(summary: str, start_time: str, end_time: str, attendees: List[str], location: str = "") -> dict:
    """
    Schedules a meeting in Google Calendar.
    This would be an API call to create a new calendar event.
    """
    print(f"DEBUG: Calling schedule_meeting_tool for {summary} at {start_time} to {end_time} with {attendees}")
    # In a real scenario, this would interact with Google Calendar API to create an event.
    # For now, just print and return a success message.
    return {
        "status": "success",
        "message": f"Meeting '{summary}' scheduled from {start_time} to {end_time}.",
        "scheduled_event_details": {
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(attendees),
            "Attendees": attendees,
            "Summary": summary
        }
    }

# Tool 3: Conflict Resolver / Rescheduler (Hypothetical)
def resolve_conflict_tool(meeting_summary: str, conflicted_time: str, alternative_times: List[str]) -> dict:
    """
    Attempts to reschedule a meeting to resolve conflicts.
    This would involve checking alternative_times and updating the calendar.
    """
    print(f"DEBUG: Resolving conflict for {meeting_summary} at {conflicted_time} with alternatives: {alternative_times}")
    # Logic to find the first available alternative and update.
    # For this example, we'll just pick the first alternative
    if alternative_times:
        new_time = alternative_times[0]
        return {"status": "resolved", "new_time": new_time, "message": f"Meeting rescheduled to {new_time}"}
    return {"status": "failed", "message": "No alternative times provided."}


tools_list = [fetch_calendar_events_tool, schedule_meeting_tool, resolve_conflict_tool]
llm_with_tools = llm.bind_tools(tools_list)

# --- 3. Define the LLM Interaction ---
def call_llm(state: AgentState) -> dict:
    """
    Calls the vLLM server with the Qwen 30B model to get a response.
    The prompt engineering here is crucial for agentic behavior.
    """
    print("DEBUG: Calling LLM for reasoning and action selection...")

    # Construct the prompt based on the current state
    # This is where you engineer your prompt to encourage reasoning, tool use, and structured output.
    # The prompt should guide the LLM to identify intent, extract entities, and decide on actions.

    # Format current calendar events for LLM
    formatted_calendar_events = []
    
    if state['calendar_events']:
        for event in state['calendar_events']:
            formatted_calendar_events.append(f"- Summary: {event.get('Summary', 'N/A')}, Start: {event.get('StartTime', 'N/A')}, End: {event.get('EndTime', 'N/A')}, Attendees: {', '.join(event.get('Attendees', []))}")
    
    calendar_info = "\nExisting Calendar Events (if any, for all attendees):\n" + "\n".join(formatted_calendar_events) if formatted_calendar_events else "\nExisting Calendar Events: None."

    prompt = f"""You are an intelligent AI scheduling assistant. Your goal is to autonomously schedule, reschedule, and optimize meetings.
    
Here is the current meeting request:
Request ID: {state['request_id']}
From: {state['sender_email']}
Attendees: {', '.join(state['attendees_emails'])}
Subject: {state['subject']}
Content: {state['email_content']}
{calendar_info}

You need to decide the next best action.
Based on the email content, determine the meeting's purpose, desired duration, and any time preferences (e.g., "Thursday").
If you need to fetch calendar availability, use the 'fetch_calendar_events_tool'. 
IMPORTANT NOTE: Fetch the calendar events for ALL the users mentioned in the attendees list along with the Sender.

If you have identified a suitable time, use the 'schedule_meeting_tool'. Ensure the scheduled time avoids conflicts with existing 'calendar_events'. If a conflict exists, propose an alternative time within the next few days.
If there are conflicts and an alternative is suggested, use 'resolve_conflict_tool'.

Your output should be a JSON object with an 'action' key and 'arguments' key.
Possible actions: "fetch_calendar_events", "schedule_meeting", "resolve_conflict", "final_answer".

Example for fetching calendar events:
{{
    "action": "fetch_calendar_events",
    "arguments": {{
        "user_email": "user@example.com",
        "start_date": "YYYY-MM-DD",
        "end_date": "YYYY-MM-DD"
    }}
}}

Example for scheduling a meeting:
{{
    "action": "schedule_meeting",
    "arguments": {{
        "summary": "Meeting Title",
        "start_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "end_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "attendees": ["email1@example.com", "email2@example.com"],
        "location": "Optional Location"
    }}
}}

Example for resolving a conflict:
{{
    "action": "resolve_conflict",
    "arguments": {{
        "meeting_summary": "Original Meeting Title",
        "conflicted_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "alternative_times": ["YYYY-MM-DDTHH:MM:SS+05:30", "YYYY-MM-DDTHH:MM:SS+05:30"]
    }}
}}

Example for final answer (when meeting is scheduled or confirmed and you have all details):
{{
    "action": "final_answer",
    "arguments": {{
        "event_start": "YYYY-MM-DDTHH:MM:SS+05:30",
        "event_end": "YYYY-MM-DDTHH:MM:SS+05:30",
        "duration_mins": "30",
        "summary": "Agentic AI Project Status Update"
    }}
}}

Consider the current date as {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.
If the email mentions "Thursday", assume it refers to the upcoming Thursday from the current date.
If the email mentions a duration (e.g., "30 minutes"), use that for the meeting.
What is your next action?
"""

    # headers = {"Content-Type": "application/json"}
    # # Adjust this URL to your vLLM server endpoint
    # vllm_url = "http://localhost:4000/v1/" #
    
    # payload = {
    #     "prompt": prompt,
    #     "max_tokens": 500,  # Adjust as needed
    #     "temperature": 0.1,
    #     "stop": ["}}", "```"] # Adjust stop tokens if your LLM output format is different
    # }

    try:
        # response = requests.post(vllm_url, headers=headers, json=payload, timeout=8) # 8-second timeout for LLM
        # response.raise_for_status()

        llm_output = llm.invoke(prompt)
        
        #llm_output = response.json()["text"][0].strip()
        
        print(f"DEBUG: LLM Raw Output: {llm_output}")

        # Attempt to parse the LLM's JSON output
        # json_start = llm_output.find('{')
        # json_end = llm_output.rfind('}')
        # if json_start != -1 and json_end != -1:
        #     json_str = llm_output[json_start : json_end + 1]
        #     llm_decision = json.loads(json_str)
        # else:
        #     raise ValueError("LLM output is not a valid JSON structure.")
        return {"llm_response": llm_output}
        # return {"llm_response": llm_output, "llm_decision": llm_decision}

    except requests.exceptions.RequestException as e:
        print(f"Error calling vLLM server: {e}")
        return {"llm_response": f"Error: {e}", "llm_decision": {"action": "error"}}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response JSON: {e}")
        return {"llm_response": f"Error parsing JSON: {e}", "llm_decision": {"action": "error"}}
    except ValueError as e:
        print(f"Error: {e}")
        return {"llm_response": f"Error: {e}", "llm_decision": {"action": "error"}}


# --- 4. Define Agent Nodes (Actions) ---

def tool_node(state: AgentState) -> AgentState:
    """
    Executes the tool chosen by the LLM.
    """
    llm_decision = state.get("llm_decision")
    action = llm_decision.get("action")
    arguments = llm_decision.get("arguments", {})
    tool_output = None
    
    updated_state = {}

    if action == "fetch_calendar_events":
        all_attendee_emails = list(set(state['attendees_emails'] + [state['sender_email']])) # Include sender
        all_attendee_events = []
        today = datetime.now()
        end_date = today + timedelta(days=14) # Fetch for the next 14 days to give more scheduling flexibility
        
        for email in all_attendee_emails:
            # Note: retrive_calendar_events expects datetime strings, not just dates.
            # Using ISO format with UTC offset for start and end times to be safe.
            # For simplicity, we'll use start of day and end of day for the range.
            start_of_day = datetime(today.year, today.month, today.day, 0, 0, 0)
            end_of_day = datetime(end_date.year, end_date.month, end_date.day, 23, 59, 59)
            
            # This is a placeholder for actual timezone handling if needed.
            # Google Calendar API often prefers 'Z' for UTC or a specific offset.
            # Assuming current time is in +05:30 (IST) as per the overall context.
            # For `timeMin` and `timeMax` in Calendar API, a simple ISO format might suffice,
            # but providing timezone info is best.
            # Example: 2025-07-19T00:00:00+05:30
            start_date_formatted = start_of_day.isoformat(timespec='seconds') + '+05:30' # Assuming IST offset
            end_date_formatted = end_of_day.isoformat(timespec='seconds') + '+05:30' # Assuming IST offset

            events_for_email = fetch_calendar_events_tool(email, start_date_formatted, end_date_formatted)
            all_attendee_events.extend(events_for_email)

        # Merge new events with existing ones in the state, avoid duplicates
        current_events = state.get('calendar_events', [])
        for new_event in all_attendee_events:
            is_duplicate = False
            for existing_event in current_events:
                if (existing_event.get('StartTime') == new_event.get('StartTime') and
                    existing_event.get('EndTime') == new_event.get('EndTime') and
                    existing_event.get('Summary') == new_event.get('Summary') and
                    sorted(existing_event.get('Attendees', [])) == sorted(new_event.get('Attendees', []))):
                    is_duplicate = True
                    break
            if not is_duplicate:
                current_events.append(new_event)
        
        tool_output = f"Fetched calendar events: {len(all_attendee_events)} events found."
        updated_state["calendar_events"] = current_events
        updated_state["tool_output"] = tool_output
        return updated_state

    elif action == "schedule_meeting":
        schedule_result = schedule_meeting_tool(
            summary=arguments.get("summary"),
            start_time=arguments.get("start_time"),
            end_time=arguments.get("end_time"),
            attendees=arguments.get("attendees"),
            location=arguments.get("location", state.get('location', "IISc Bangalore")) # Use location from state or default
        )
        
        updated_state["tool_output"] = json.dumps(schedule_result)
        
        # Update state with scheduled event details for final output formatting
        if schedule_result.get("status") == "success" and schedule_result.get("scheduled_event_details"):
            scheduled_details = schedule_result["scheduled_event_details"]
            updated_state["event_start"] = scheduled_details.get("StartTime", "")
            updated_state["event_end"] = scheduled_details.get("EndTime", "")
            
            try:
                # Ensure correct parsing for duration calculation, handle potential missing timezone
                start_time_str = scheduled_details["StartTime"]
                end_time_str = scheduled_details["EndTime"]

                # Add a dummy timezone if not present, for parsing
                if len(start_time_str) == 19 and start_time_str.endswith('T'): # YYYY-MM-DDTHH:MM:SS
                    start_dt = datetime.fromisoformat(start_time_str + '+00:00') # Assume UTC if no TZ
                elif len(start_time_str) > 19 and (start_time_str[-6] == '+' or start_time_str[-6] == '-'): # YYYY-MM-DDTHH:MM:SS+HH:MM
                    start_dt = datetime.fromisoformat(start_time_str)
                else: # Fallback for other formats, try parsing directly
                    start_dt = datetime.fromisoformat(start_time_str.replace('Z', '+00:00')) # Replace Z with UTC offset
                
                if len(end_time_str) == 19 and end_time_str.endswith('T'):
                    end_dt = datetime.fromisoformat(end_time_str + '+00:00')
                elif len(end_time_str) > 19 and (end_time_str[-6] == '+' or end_time_str[-6] == '-'):
                    end_dt = datetime.fromisoformat(end_time_str)
                else:
                    end_dt = datetime.fromisoformat(end_time_str.replace('Z', '+00:00'))


                updated_state["duration_mins"] = str(int((end_dt - start_dt).total_seconds() / 60))
            except ValueError as e:
                print(f"Warning: Could not parse start/end times for duration calculation: {scheduled_details.get('StartTime')}, {scheduled_details.get('EndTime')} Error: {e}")
                updated_state["duration_mins"] = "0" # Default or placeholder
            
            updated_state["scheduled_event_summary"] = scheduled_details.get("Summary", "")
            updated_state["scheduled_event_attendees"] = scheduled_details.get("Attendees", [])
            
            # Add the newly scheduled event to calendar_events for the output formatting node
            current_events = state.get('calendar_events', [])
            current_events.append(scheduled_details)
            updated_state["calendar_events"] = current_events

        return updated_state

    elif action == "resolve_conflict":
        resolve_result = resolve_conflict_tool(
            meeting_summary=arguments.get("meeting_summary"),
            conflicted_time=arguments.get("conflicted_time"),
            alternative_times=arguments.get("alternative_times")
        )
        updated_state["tool_output"] = json.dumps(resolve_result)
        # If successfully resolved, the LLM should then proceed to schedule_meeting again
        return updated_state
    
    elif action == "error":
        print(f"Error action triggered: {state.get('llm_response')}")
        return {"tool_output": state.get('llm_response')}

    else:
        tool_output = f"No valid tool action specified by LLM or action: {action} is not supported. LLM Response: {state.get('llm_response')}"
        print(tool_output)
        return {"tool_output": tool_output}

def format_final_output_node(state: AgentState) -> AgentState:
    """
    Formats the final output as per the specified JSON structure for submission.
    """
    print("DEBUG: Formatting final output...")
    
    # Consolidate all relevant attendees, including the sender
    all_involved_emails = list(set(state['attendees_emails'] + [state['sender_email']]))
    
    output_attendees = []
    
    # Iterate through all involved emails to gather their events
    for email in all_involved_emails:
        attendee_data = {"email": email, "events": []}
        
        # Add events from the 'calendar_events' list that involve this attendee
        for event in state['calendar_events']:
            # Check if the attendee's email is explicitly in the event's attendees list
            # Or if 'SELF' is used and it's the sender's email
            if email in event.get('Attendees', []) or ("SELF" in event.get('Attendees', []) and email == state['sender_email']):
                 # Ensure the event structure matches the expected output
                formatted_event = {
                    "StartTime": event.get("StartTime", ""),
                    "EndTime": event.get("EndTime", ""),
                    "NumAttendees": event.get("NumAttendees", 0),
                    "Attendees": event.get("Attendees", []),
                    "Summary": event.get("Summary", "")
                }
                # Avoid adding exact duplicate events within a single attendee's list
                if formatted_event not in attendee_data["events"]:
                    attendee_data["events"].append(formatted_event)
        output_attendees.append(attendee_data)

    output = {
        "Request_id": state['request_id'],
        "Datetime": state['datetime_of_request'],
        "Location": state.get('location', "IISc Bangalore"), # Use location from state or default
        "From": state['sender_email'],
        "Attendees": output_attendees, # Updated attendees structure
        "Subject": state['subject'], # Use original subject, or scheduled_event_summary if LLM refined
        "EmailContent": state['email_content'],
        "EventStart": state.get('event_start', ""),
        "EventEnd": state.get('event_end', ""),
        "Duration_mins": state.get('duration_mins', ""),
        "MetaData": state.get('metadata', {}) # Should typically be an empty dict or specific metadata
    }

    return {"final_output": output}


# --- 5. Define Conditional Edges (Routing Logic) ---

def decide_next_step(state: AgentState) -> str:
    """
    Determines the next step in the graph based on the LLM's decision.
    """
    llm_decision = state.get("llm_decision")
    if not llm_decision:
        print("DECISION: No LLM decision found, stopping.")
        return "end" # Or transition to an error handling node

    action = llm_decision.get("action")

    if action == "fetch_calendar_events":
        print("DECISION: LLM decided to fetch calendar events.")
        return "call_tool"
    elif action == "schedule_meeting":
        print("DECISION: LLM decided to schedule meeting.")
        return "call_tool"
    elif action == "resolve_conflict":
        print("DECISION: LLM decided to resolve conflict.")
        return "call_tool"
    elif action == "final_answer":
        print("DECISION: LLM provided final answer.")
        return "format_output"
    elif action == "error":
        print("DECISION: LLM indicated an error or invalid action.")
        return "format_output" # Can also route to a specific error handling node
    else:
        print(f"DECISION: Unknown action: {action}. Re-calling LLM.")
        return "call_llm" # Loop back to LLM if decision is unclear or invalid


# --- 6. Build the LangGraph ---

def create_scheduling_agent_graph():
    workflow = StateGraph(AgentState)

    # Add nodes to the graph
    workflow.add_node("call_llm", call_llm)
    workflow.add_node("call_tool", tool_node)
    workflow.add_node("format_output", format_final_output_node)

    # Set the entry point
    workflow.set_entry_point("call_llm")

    # Define edges (transitions)
    workflow.add_conditional_edges(
        "call_llm",  # From call_llm node
        decide_next_step, # Use decide_next_step function to determine next node
        {
            "call_tool": "call_tool",
            "format_output": "format_output",
            "end": END # If LLM decides it's done or an error occurs
        }
    )
    
    # After a tool call, always go back to the LLM for further reasoning/action
    workflow.add_edge("call_tool", "call_llm")

    # After formatting the output, the process ends
    workflow.add_edge("format_output", END)

    return workflow.compile()

# Instantiate the graph
scheduling_agent_graph = create_scheduling_agent_graph()



# --- 7. Integrate with your `your_meeting_assistant` function ---

def your_meeting_assistant(data: dict):
    """
    Main function to be called by the Flask server.
    Initializes the agent state and runs the LangGraph.
    """
    # Initialize the state based on the input request JSON
    initial_state = AgentState(
        request_id=data.get("Request_id", ""),
        datetime_of_request=data.get("Datetime", ""),
        sender_email=data.get("From", ""),
        attendees_emails=[att["email"] for att in data.get("Attendees", [])],
        subject=data.get("Subject", ""),
        email_content=data.get("EmailContent", ""),
        location=data.get("Location", "IISc Bangalore"), # Initialize location
        calendar_events=[],  # Will be populated by the agent
        event_start="",
        event_end="",
        duration_mins="",
        metadata=data.get("MetaData", {}), # Initialize with any metadata from input
        intermediate_steps=[],
        tool_output="",
        llm_response="",
        scheduled_event_summary="",
        scheduled_event_attendees=[]
    )

    print(scheduling_agent_graph.invoke(initial_state))

    
    # # Run the graph
    # final_state = None
    # for s in scheduling_agent_graph.stream(initial_state, {"recursion_limit": 50}): # Limit recursion to prevent infinite loops
    #     # print(s) # Uncomment for debugging to see intermediate states
    #     final_state = s

    # # Extract the final output from the last state
    # if final_state and 'format_output' in final_state:
    #     return final_state['format_output']['final_output']
    # elif final_state and 'llm_decision' in final_state and final_state['llm_decision'].get('action') == 'final_answer':
    #      # If LLM directly returned a final answer without going through format_output_node
    #     llm_args = final_state['llm_decision'].get('arguments', {})
    #     # This path might not generate the full 'events' list per attendee,
    #     # so it's best to always route through `format_final_output_node` for consistency.
    #     # However, for robustness, we'll try to construct a basic one if this path is hit.
        
    #     # Manually construct simplified output for this direct LLM path
    #     output_attendees_simplified = []
    #     all_involved_emails = list(set(initial_state['attendees_emails'] + [initial_state['sender_email']]))
    #     for email in all_involved_emails:
    #         output_attendees_simplified.append({"email": email, "events": []}) # Events might be empty here
        
    #     output_from_llm_direct = {
    #         "Request_id": initial_state['request_id'],
    #         "Datetime": initial_state['datetime_of_request'],
    #         "Location": initial_state.get('location', "IISc Bangalore"),
    #         "From": initial_state['sender_email'],
    #         "Attendees": output_attendees_simplified, # Simplified attendees
    #         "Subject": llm_args.get("summary", initial_state['subject']),
    #         "EmailContent": initial_state['email_content'],
    #         "EventStart": llm_args.get("event_start", ""),
    #         "EventEnd": llm_args.get("event_end", ""),
    #         "Duration_mins": llm_args.get("duration_mins", ""),
    #         "MetaData": initial_state.get('metadata', {})
    #     }
    #     return output_from_llm_direct
    # else:
    #     print("ERROR: Graph did not produce a final formatted output.")
    #     # Return a default or error output format
    #     output_attendees_error = []
    #     all_involved_emails = list(set(initial_state['attendees_emails'] + [initial_state['sender_email']]))
    #     for email in all_involved_emails:
    #         output_attendees_error.append({"email": email, "events": []})

    #     return {
    #         "Request_id": initial_state['request_id'],
    #         "Datetime": initial_state['datetime_of_request'],
    #         "Location": initial_state.get('location', "IISc Bangalore"),
    #         "From": initial_state['sender_email'],
    #         "Attendees": output_attendees_error,
    #         "Subject": initial_state['subject'],
    #         "EmailContent": initial_state['email_content'],
    #         "EventStart": "",
    #         "EventEnd": "",
    #         "Duration_mins": "",
    #         "MetaData": {"error": "Could not schedule meeting or format output. Check LLM logs."}
    #     }

    

## Testing JSON 

In [37]:
import json 

json_data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

your_meeting_assistant(json_data)

DEBUG: Calling LLM for reasoning and action selection...
DEBUG: LLM Raw Output: content='To determine the next best action, I need to analyze the meeting request.\n\nMeeting Request:\n- Request ID: 6118b54f-907b-4451-8d48-dd13d76033a5\n- From: userone.amd@gmail.com\n- Attendees: usertwo.amd@gmail.com, userthree.amd@gmail.com\n- Subject: Agentic AI Project Status Update\n- Content: Hi team, let\'s meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.\n\nBased on the email content, I can infer the following:\n- Meeting purpose: Status update of the Agentic AI Project\n- Desired duration: 30 minutes\n- Time preference: Thursday (upcoming Thursday from the current date)\n\nTo find a suitable time for the meeting, I need to fetch the calendar events for all the users mentioned in the attendees list along with the sender.\n\nNext Action:\n{\n    "action": "fetch_calendar_events",\n    "arguments": {\n        "user_emails": ["userone.amd@gmail.com", "usertwo.amd@gmail.c

## Without Markdown

In [38]:
from typing import List
import json
import requests
from datetime import datetime, timedelta

# Import necessary Google API client libraries - assuming they are installed
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# --- Define Tools (Functions the AI Agent can use) ---

# Tool 1: Google Calendar Event Fetcher Wrapper
def fetch_calendar_events_tool(attendee_email: str, start_date_str: str, end_date_str: str) -> List[dict]:
    """
    Fetches calendar events for a given attendee within a date range using the provided retrieval function.
    """
    print(f"DEBUG: Calling fetch_calendar_events_tool for {attendee_email} from {start_date_str} to {end_date_str}")
    
    events_list = []
    token_path = "Keys/"+attendee_email.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')

    count=0
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            events_list.append(
                {"StartTime" : start_time, 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(attendee_list)), 
                 "Attendees" : list(set(attendee_list)),
                 "Summary" : event["summary"]})
        except Exception as E:
            count+=1
    print('No of exceptions are: ',count)
    return events_list

# Tool 2: Meeting Scheduler (Hypothetical - this would interact with Google Calendar API to create an event)
def schedule_meeting_tool(summary: str, start_time: str, end_time: str, attendees: List[str], location: str = "") -> dict:
    """
    Schedules a meeting in Google Calendar.
    This would be an API call to create a new calendar event.
    """
    print(f"DEBUG: Calling schedule_meeting_tool for {summary} at {start_time} to {end_time} with {attendees}")
    # In a real scenario, this would interact with Google Calendar API to create an event.
    # For now, just print and return a success message.
    return {
        "status": "success",
        "message": f"Meeting '{summary}' scheduled from {start_time} to {end_time}.",
        "scheduled_event_details": {
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(attendees),
            "Attendees": attendees,
            "Summary": summary
        }
    }

# Tool 3: Conflict Resolver / Rescheduler (Hypothetical)
def resolve_conflict_tool(meeting_summary: str, conflicted_time: str, alternative_times: List[str]) -> dict:
    """
    Attempts to reschedule a meeting to resolve conflicts.
    This would involve checking alternative_times and updating the calendar.
    """
    print(f"DEBUG: Resolving conflict for {meeting_summary} at {conflicted_time} with alternatives: {alternative_times}")
    # Logic to find the first available alternative and update.
    # For this example, we'll just pick the first alternative
    if alternative_times:
        new_time = alternative_times[0]
        return {"status": "resolved", "new_time": new_time, "message": f"Meeting rescheduled to {new_time}"}
    return {"status": "failed", "message": "No alternative times provided."}


# --- LLM Interaction Function ---
def call_llm_for_decision(prompt: str) -> dict:
    """
    Calls the vLLM server to get a response and parse the JSON decision.
    """
    print("DEBUG: Calling LLM for reasoning and action selection...")

    # headers = {"Content-Type": "application/json"}
    # vllm_url = "http://localhost:8000/v1" # Adjust this URL to your vLLM server endpoint if different
    
    # payload = {
    #     "prompt": prompt,
    #     "max_tokens": 500,  # Adjust as needed
    #     "temperature": 0.1,
    #     "stop": ["}}", "```"] # Adjust stop tokens if your LLM output format is different
    # }

    try:

        # response = llm.in
        
        # response = requests.post(vllm_url, headers=headers, json=payload, timeout=8) # 8-second timeout for LLM
        # response.raise_for_status()
        llm_output = llm.invoke(prompt)
        print(f"DEBUG: LLM Raw Output: {llm_output}")

        # Attempt to parse the LLM's JSON output
        json_start = llm_output.find('{')
        json_end = llm_output.rfind('}')
        if json_start != -1 and json_end != -1:
            json_str = llm_output[json_start : json_end + 1]
            llm_decision = json.loads(json_str)
            return {"llm_response": llm_output, "llm_decision": llm_decision}
        else:
            raise ValueError("LLM output is not a valid JSON structure.")

    except requests.exceptions.RequestException as e:
        print(f"Error calling vLLM server: {e}")
        return {"llm_response": f"Error: {e}", "llm_decision": {"action": "error", "message": f"vLLM server error: {e}"}}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response JSON: {e}")
        return {"llm_response": f"Error parsing JSON: {e}", "llm_decision": {"action": "error", "message": f"JSON parsing error: {e}"}}
    except ValueError as e:
        print(f"Error: {e}")
        return {"llm_response": f"Error: {e}", "llm_decision": {"action": "error", "message": f"ValueError: {e}"}}


# --- Main Assistant Logic (replaces LangGraph) ---

def your_meeting_assistant(data: dict) -> dict:
    """
    Main function to process meeting requests, fetch calendar data,
    use LLM for decision making, and schedule/resolve conflicts.
    """
    request_id = data.get("Request_id", "")
    datetime_of_request = data.get("Datetime", "")
    sender_email = data.get("From", "")
    attendees_emails = [att["email"] for att in data.get("Attendees", [])]
    subject = data.get("Subject", "")
    email_content = data.get("EmailContent", "")
    location = data.get("Location", "IISc Bangalore")
    metadata = data.get("MetaData", {})

    all_involved_emails = list(set(attendees_emails + [sender_email]))

    # --- Step 1: Asynchronously (sequentially in this context) fetch all calendar events ---
    all_attendee_events = []
    today = datetime.now()
    # Fetch events for a reasonable future period, e.g., next 14 days
    end_date_fetch = today + timedelta(days=14) 
    
    # Format dates for Google Calendar API (ISO 8601 with offset)
    # Ensuring +05:30 IST timezone for all date/time operations as requested
    current_time_offset = "+05:30" 
    start_of_day_str = datetime(today.year, today.month, today.day, 0, 0, 0).isoformat(timespec='seconds') + current_time_offset
    end_of_period_str = datetime(end_date_fetch.year, end_date_fetch.month, end_date_fetch.day, 23, 59, 59).isoformat(timespec='seconds') + current_time_offset

    for email in all_involved_emails:
        events_for_email = fetch_calendar_events_tool(email, start_of_day_str, end_of_period_str)
        all_attendee_events.extend(events_for_email)
    
    # Deduplicate the combined list of events
    unique_combined_events = []
    seen_events_set = set()
    for event in all_attendee_events:
        event_key = (event.get('StartTime'), event.get('EndTime'), event.get('Summary'), tuple(sorted(event.get('Attendees', []))))
        if event_key not in seen_events_set:
            unique_combined_events.append(event)
            seen_events_set.add(event_key)
    
    calendar_events = unique_combined_events

    # --- Step 2: Prepare prompt for LLM with fetched calendar data ---
    formatted_calendar_events = []
    if calendar_events:
        for event in calendar_events:
            formatted_calendar_events.append(f"- Summary: {event.get('Summary', 'N/A')}, Start: {event.get('StartTime', 'N/A')}, End: {event.get('EndTime', 'N/A')}, Attendees: {', '.join(event.get('Attendees', []))}")
    
    calendar_info = "\nExisting Calendar Events (for all attendees and sender):\n" + "\n".join(formatted_calendar_events) if formatted_calendar_events else "\nExisting Calendar Events: None."

    prompt = f"""You are an intelligent AI scheduling assistant. Your goal is to autonomously schedule, reschedule, and optimize meetings.
    
Here is the current meeting request:
Request ID: {request_id}
From: {sender_email}
Attendees: {', '.join(attendees_emails)}
Subject: {subject}
Content: {email_content}
{calendar_info}

Current Date and Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

You need to analyze the request, the attendees' calendars, and decide the next best action.
**Instructions for your decision:**
1.  **Extract Meeting Details:** Determine the meeting's purpose (summary), desired duration, and any time preferences (e.g., "Thursday", "10:00 A.M."). If "Thursday" is mentioned, assume it's the upcoming Thursday from the current date.
2.  **System Outage Priority:** If the **Subject** or **Content** explicitly mentions "system outage" or other critical, immediate issues, you must prioritize scheduling the meeting **immediately**, even if it causes a conflict. In such cases, propose the earliest possible slot.
3.  **Conflict Resolution & Innovation:**
    * Carefully analyze the provided "Existing Calendar Events" for all attendees to find a free slot.
    * If a conflict exists, propose alternative times that work for all attendees. Be innovative in finding a suitable slot, considering a few days into the future.
    * If the request mentions "important call" or "off-hours" preferences, try to accommodate them.
4.  **Action Selection:** Your response must be a JSON object with an 'action' key and an 'arguments' key.

Possible actions: "schedule_meeting", "resolve_conflict", "final_answer".

Example for scheduling a meeting:
{{
    "action": "schedule_meeting",
    "arguments": {{
        "summary": "Meeting Title",
        "start_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "end_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "attendees": ["email1@example.com", "email2@example.com"],
        "location": "Optional Location"
    }}
}}

Example for resolving a conflict:
{{
    "action": "resolve_conflict",
    "arguments": {{
        "meeting_summary": "Original Meeting Title",
        "conflicted_time": "YYYY-MM-DDTHH:MM:SS+05:30",
        "alternative_times": ["YYYY-MM-DDTHH:MM:SS+05:30", "YYYY-MM-DDTHH:MM:SS+05:30"]
    }}
}}

Example for final answer (when meeting is scheduled or confirmed and you have all details):
{{
    "action": "final_answer",
    "arguments": {{
        "event_start": "YYYY-MM-DDTHH:MM:SS+05:30",
        "event_end": "YYYY-MM-DDTHH:MM:SS+05:30",
        "duration_mins": "30",
        "summary": "Agentic AI Project Status Update"
    }}
}}

**Based on the request and the fetched calendar events, what is your next action and its arguments?**
"""

    llm_result = call_llm_for_decision(prompt)
    llm_decision = llm_result.get("llm_decision", {"action": "error", "message": "No decision from LLM."})
    action = llm_decision.get("action")
    arguments = llm_decision.get("arguments", {})

    final_event_start = ""
    final_event_end = ""
    final_duration_mins = ""
    final_summary = subject # Default to original subject

    # --- Step 3: Execute action based on LLM's decision ---
    if action == "schedule_meeting":
        schedule_result = schedule_meeting_tool(
            summary=arguments.get("summary", subject),
            start_time=arguments.get("start_time"),
            end_time=arguments.get("end_time"),
            attendees=arguments.get("attendees", all_involved_emails),
            location=arguments.get("location", location)
        )
        if schedule_result.get("status") == "success":
            scheduled_details = schedule_result["scheduled_event_details"]
            final_event_start = scheduled_details.get("StartTime", "")
            final_event_end = scheduled_details.get("EndTime", "")
            final_summary = scheduled_details.get("Summary", subject)
            
            try:
                # Calculate duration, handling potential missing timezone for parsing
                start_dt = datetime.fromisoformat(final_event_start.replace('Z', '+00:00'))
                end_dt = datetime.fromisoformat(final_event_end.replace('Z', '+00:00'))
                final_duration_mins = str(int((end_dt - start_dt).total_seconds() / 60))
            except ValueError:
                final_duration_mins = "0" # Could not parse times
            
            # Add the newly scheduled event to calendar_events for the final output's attendee events
            calendar_events.append(scheduled_details)

    elif action == "resolve_conflict":
        resolve_result = resolve_conflict_tool(
            meeting_summary=arguments.get("meeting_summary", subject),
            conflicted_time=arguments.get("conflicted_time"),
            alternative_times=arguments.get("alternative_times", [])
        )
        
        if resolve_result.get("status") == "resolved":
            print(f"DEBUG: Conflict resolved, new time suggested: {resolve_result.get('new_time')}")
            # If a conflict is resolved, we immediately attempt to schedule at the new time.
            # We need to derive the duration from the original request or a default.
            # For simplicity, let's assume a default duration if not clearly extracted by LLM
            # In a real scenario, LLM should extract duration consistently.
            duration_from_request = 30 # This should ideally be extracted robustly from EmailContent by LLM
            
            resolved_start_time = resolve_result.get('new_time')
            if resolved_start_time:
                try:
                    # Calculate end time based on new start time and assumed duration
                    # Need to be careful with timezone here for accurate calculation
                    start_dt_resolved = datetime.fromisoformat(resolved_start_time.replace('Z', '+00:00'))
                    resolved_end_time = (start_dt_resolved + timedelta(minutes=duration_from_request)).isoformat(timespec='seconds') + current_time_offset

                    schedule_after_resolve_result = schedule_meeting_tool(
                        summary=arguments.get("meeting_summary", subject), # Use original summary
                        start_time=resolved_start_time,
                        end_time=resolved_end_time,
                        attendees=arguments.get("attendees", all_involved_emails), # LLM should provide attendees
                        location=arguments.get("location", location) # LLM should provide location
                    )
                    if schedule_after_resolve_result.get("status") == "success":
                        scheduled_details = schedule_after_resolve_result["scheduled_event_details"]
                        final_event_start = scheduled_details.get("StartTime", "")
                        final_event_end = scheduled_details.get("EndTime", "")
                        final_summary = scheduled_details.get("Summary", subject)
                        final_duration_mins = str(int((datetime.fromisoformat(final_event_end.replace('Z', '+00:00')) - datetime.fromisoformat(final_event_start.replace('Z', '+00:00'))).total_seconds() / 60))
                        calendar_events.append(scheduled_details) # Add new event
                except ValueError as e:
                    print(f"ERROR: Could not parse resolved time or calculate end time: {e}")
                    metadata["error"] = f"Failed to schedule after conflict resolution: {e}"
            else:
                metadata["error"] = "Conflict resolved but no new time provided to schedule."
        else:
            metadata["error"] = "Conflict resolution failed."
        
    elif action == "final_answer":
        # LLM directly provided a final answer without needing scheduling
        final_event_start = arguments.get("event_start", "")
        final_event_end = arguments.get("event_end", "")
        final_duration_mins = arguments.get("duration_mins", "")
        final_summary = arguments.get("summary", subject)
        
    elif action == "error":
        print(f"Error action from LLM: {llm_decision.get('message')}")
        # Fallback for error or unrecognized action
        final_event_start = ""
        final_event_end = ""
        final_duration_mins = ""
        final_summary = subject
        metadata["error"] = llm_decision.get('message', "LLM could not determine a valid action.")
        
    else:
        print(f"DEBUG: LLM returned an unexpected action: {action}. Defaulting to no scheduling.")
        final_event_start = ""
        final_event_end = ""
        final_duration_mins = ""
        final_summary = subject
        metadata["error"] = f"LLM returned an unexpected action: {action}"


    # --- Step 4: Format Final Output ---
    output_attendees_structured = []
    for email in all_involved_emails:
        attendee_data = {"email": email, "events": []}
        for event in calendar_events:
            # Check if the event's attendees list contains the current email
            if email in event.get('Attendees', []):
                formatted_event = {
                    "StartTime": event.get("StartTime", ""),
                    "EndTime": event.get("EndTime", ""),
                    "NumAttendees": event.get("NumAttendees", 0),
                    "Attendees": event.get("Attendees", []),
                    "Summary": event.get("Summary", "")
                }
                # Prevent duplicate entries for the same event if it appears multiple times
                if formatted_event not in attendee_data["events"]:
                    attendee_data["events"].append(formatted_event)
        output_attendees_structured.append(attendee_data)

    final_output = {
        "Request_id": request_id,
        "Datetime": datetime_of_request,
        "Location": location,
        "From": sender_email,
        "Attendees": output_attendees_structured,
        "Subject": subject,
        "EmailContent": email_content,
        "EventStart": final_event_start,
        "EventEnd": final_event_end,
        "Duration_mins": final_duration_mins,
        "MetaData": metadata
    }

    return final_output

data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

response_data = your_meeting_assistant(data)

print(response_data)

DEBUG: Calling fetch_calendar_events_tool for usertwo.amd@gmail.com from 2025-07-19T00:00:00+05:30 to 2025-08-02T23:59:59+05:30


FileNotFoundError: [Errno 2] No such file or directory: 'Keys/usertwo.amd.token'

In [31]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')

    count=0
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        try:
            start_time = event["start"]["dateTime"]
            end_time = event["end"]["dateTime"]
            events_list.append(
                {"StartTime" : start_time, 
                 "EndTime": end_time, 
                 "NumAttendees" :len(set(attendee_list)), 
                 "Attendees" : list(set(attendee_list)),
                 "Summary" : event["summary"]})
        except Exception as E:
            count+=1
    print('No of exceptions are: ',count)
    return events_list

In [32]:
event = retrive_calendar_events("userone.amd@gmail.com", '2023-07-17T00:00:00+05:30', '2026-07-17T23:59:59+05:30')

FileNotFoundError: [Errno 2] No such file or directory: 'Keys/userone.amd.token'

## Deploying model for Inference

In [9]:
# --- Flask Server Integration (from your provided notebook) ---

from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\nReceived: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data) # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\nSending: {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='129.212.190.146', port=5002)

# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5002 is in use by another program. Either identify and stop that program, or start the server with a different port.
